In [1]:
from IPython.core.display import HTML

HTML("""
<style>
    p {
        font-size: 1.2em;
        line-height: 1.5em;
    }
</style>
""")

# Making the Right Choice - A/B Testing

This notebook covers:
- A/B testing
- Complexities when making the right choice
- Multi-armed bandits

Given a number of options, how do we choose the one that maximizes our reward (or, equivalently, minimizes the cost)? For example, if we have two possible routes to work, how might we choose the one that minimizes the time spent traveling? In this example, our reward is based on the time it takes to get to work but could equally be the cost of fuel or the time spent in traffic.

Any problem whereby an option is tested multiple times, and where each time an option is chosen a reward is returned, can be optimized using the techniques in this notebook. In this example, the route to work is decided every day, and we can record the length of the commute in a ledger. Over time, our commuter may discover patterns in the data (route A takes less time than route B) and choose this one consistently. What then is a good strategy for the commuter to take? To consistently take route A or route B? When might they have enough data to decide which route is best? What is the optimal strategy for testing these routes? These questions are the focus of this notebook. The figure provides a graphical overview of the problem definition

*Formalizing the problem definition. In this situation, our actor is faced with a decision, and, depending on their choice, they reach an outcome. This outcome has an associated reward that is returned to the actor. Given that the user is exposed to this decision continually, how do they formulate a strategy to obtain the maximum reward (or, equivalently, the minimum cost)?*

![image](images/07fig01.jpg)

In this figure, we assume that the actor is exposed many times to a situation that requires a decision to be made. Each time a decision is made, an outcome is reached, and this outcome has an associated reward associated with it. In the case of our commuter, assuming they go to work every day, they have to choose their route every time they go to work. The outcome is the combination of all the factors on that commute, and the reward is formulated from this.

Although our toy example fits the problem perfectly, the intelligent web offers many other problems with the same format. It offers many applications where we’re presented with several choices and making the correct one is crucial:

- _Landing-page optimization_— With the advent of the intelligent web, there are many more digital-only retailers. For such retailers, ensuring maximum conversion rate (the rate at which people who visit the website go on to buy products or interact with pages deeper in the site) is of key concern. The decision to be made regards the format and content of the landing page (we may have a selection of three or four to choose from). We’d like to choose the best landing page from our set that grabs the incoming users and maximizes their likelihood to interact or to buy.

- _Ad creative optimization_— As you may have realized, online advertising provides a host of challenges that are appropriate for the application of machine-learning techniques. One of the more interesting challenges focuses on the selection of the format of the ad or aspects of the ad. Once we’ve decided that we want to show an ad and what price we wish to pay for it, what do we place within that space? We could test for a host of decisions. Choosing the right combination of aspects could result in an ad that performs better than those with other combinations.


## A/B testing

So how do we evaluate decisions, and what strategy should we employ to test our outcomes? One example involves A/B testing. Within industry, A/B testing has gained popularity in recent years, but those with a background in statistics will recognize it simply as a two-group experiment. Let’s delve a little deeper to understand the specifics.

When performing A/B testing, we do so with two groups: group A and group B. The first is a control group, and the second has some factor changed. For example, in the case of landing-page optimization, group A may be shown the existing landing page and group B a new landing page whose content or layout has changed in some way. The purpose of our A/B test is to understand whether the new layout changes the conversion rate in a statistically significant way.

It’s worth noting that the assignment of users to their respective groups requires some consideration. For A/B testing, we effectively randomize assignment, so for a large number of users the cross section of the population should be the same (that is, no biases within the groups). Take care, though; for small groups of users, this won’t be the case, and so other experimental designs should be considered.

### The Theory

Given that we’ve constructed two large groups of users who differ only by the landing page they’ve arrived at, we now wish to test whether there is a statistically significant difference between the conversion rates of the two groups. Due to the large sample size, we can do this using a two-sample, one-tailed z-test, but for smaller sets we may rely on the t-test.

The z-test operates under the assumption that the data is normally distributed and randomly sampled. Our aim is to test whether our test set (group B) is significantly different than that of this control set (group A). But how is this test performed?

Let’s assume that we have 5,000 samples from each of group A and group B. We need a mathematical formulation to state our null hypothesis—that there is no significant positive difference in the population conversion rates of the two groups—and an alternative hypothesis that there is indeed a significant positive difference between the population conversion rates.

We can think of the sampled conversion rate as a normally distributed random variable. That is, the sampled conversion rate is an observation from a normal distribution over conversion rates. To understand this, consider that multiple experiments drawn from the same group will result in slightly different conversion rates. Each time we sample the group, we obtain an estimate of the population conversion rate. This is true of both groups A/B. We can therefor come up with a new, also normal, random variable that’s the combination of random variables from groups A and B. This is the distribution of differences. Let us write $X$ to refer to this new random variable, defined as

$X$ = $X_e$ – $X_n$

where $X_e$ is the random variable of conversion rates of our experimental group, and $X_n$ is the random variable of conversion rates of our control group. Given this new random variable, we can now write the null and alternative hypothesis. Our null hypothesis can be stated as follows:

$H_0$: $X$ = $0$

That is, the experimental group is no different than the control group. Both random variables $X_e$ and $X_n$ are distributed around the same population mean, so our new random variable $X$ should be distributed around $0$. Our alternative hypothesis can be stated as follows:

$H_a$: $X$ > $0$

That is, the expectation of the random variable of the experimental group is larger than that of the control group; the population mean of this group is higher.

We can perform a one-tailed z-test on the distribution of $X$, under the assumption of the null hypothesis, to determine whether there is evidence to support the alternative hypothesis. To perform this, we sample $X$, calculate the standard score, and test against known significance levels.

Sampling of $X$ is equivalent to running the two experiments, determining their respective conversion rates, and subtracting the control conversion rate from the experimental one. From the definition of the standard score, we may now write

$z$ = ($p_{experiment}$ – $p_{control}$)/ $SE$

where $p_{experiment}$ is the conversion rate of the experiment, $p_{control}$ is the conversion rate of the control, and $SE$ is the standard error for the difference in conversion rates.

To determine the standard error, we note that conversions are binomially distributed, so visits to the site can be seen as a single Bernoulli trial with an unknown chance of a positive outcome (a conversion). Provided the number of samples is large enough, we can, however, approximate this distribution as a normal using the widely adopted Wald method. To capture the uncertainty of the particular conversion rate, we can write the standard error ($SE$) for both the experimental and control groups as follows, where $p$ is the likelihood of a conversion and $n$ is the number of samples:

![image](images/165equ01.jpg)

The numerator derives from the the variance of the binomial distribution ($np$(1 – $p$)), and the denominator captures the fact that the error in conversion rate will drop the more samples that are taken. Noting that the probability of a positive outcome is equivalent to the conversion rate, and because the standard error of two variables can be combined by addition, we note the following:

![image](images/166equ01.jpg)

By substitution, we can now write our z-test as follows. This is a formulation of the Wald (or normal) interval for the binomial distribution:

![image](images/166equ02.jpg)


The greater the value of $z$, the more evidence against the null hypothesis. To obtain a 90% confidence interval for a one-tail test, our value of $z$ would need to be greater than 1.28. What this actually says is that, under the assumption of the null hypothesis (the population mean for groups A and B is the same), the probability of this difference in conversion rates, or one larger than this, occurring by chance is less than 10%. Put another way, assuming the control and experiment conversion rates are drawn from a distribution with the same mean, if we run this same experiment 100 times, only 10 of these would have at least such an extreme value. We can provide even tighter bounds and more evidence against the null hypothesis by using a 95% confidence interval. This increases the $z$ value required to 1.65.

It might be useful for you to think about the factors that will impact the size of $z$. Obviously, if we draw two conversion rates at a given point in time from an experimental set and a control set, a larger difference in these will lead to a larger $z$ score, and therefore more evidence that they’re drawn from different populations with different means. But the number of samples is also important. As you’ve seen, a larger number of samples will lead to an overall smaller standard error. This captures the fact that our estimate of conversion rate is more accurate the longer we run the experiment. In the following section, we’ll provide an illustration of this and a worked example of A/B testing in Python using this methodology.

### The Code

Imagine that you’re in charge of a large retail site and that your design team has just changed the landing page. You receive approximately 20,000 users per week and can quantify the conversion rate of your users: that is, what percentage goes on to buy a product. The design team assures you that the new site will drive more customers, but you aren’t so sure and want to run an A/B test to see if performance will increase.

Entering users are randomly assigned to group A or group B the first time they’re seen and remain in this group for the duration of the experiment. The average conversion rate of the users is assessed within the two groups at the end of the experiment and found to be 0.002 for the new landing page and 0.001 for the original landing page. You need to know whether this increase is significant enough to warrant the landing page being permanently changed to the new design. Let’s look at the code that will help you answer this question.

In [3]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import itertools

#Assume 5000 samples per group
n_experiment = 10000
n_control = 10000

p_experiment= 0.002
p_control = 0.001

se_experiment_sq = p_experiment*(1-p_experiment) / n_experiment
se_control_sq = p_control*(1-p_control) / n_control

Z = (p_experiment-p_control)/math.sqrt(se_experiment_sq+se_control_sq)

print(Z)

1.8272652143588817


This code obtains the $z$ value for the experiment. Given these values, we obtain a $z$ value of 1.827. This exceeds the 92% confidence interval but not the 95% interval. We can say that there is a probability less than 0.08 that the data was drawn from the control distribution. Consequently, at this interval the data is significant. We should reject the null hypothesis that there is no difference at all between the groups and accept the alternative hypothesis, that the second group has a higher conversion rate. If we’ve controlled all other aspects of the user groups, this should imply that the website redesign has had a positive effect.

You should be able to see from this code that the standard error of the distributions from which the conversion rate is drawn has a direct effect on the $z$ values returned. The higher the $SE$ of the two groups, the smaller the value of $z$ and therefore the less significant the result, given constant values of $p_{experiment}$ and $p_{control}$. We also note that, because of the definition of $SE$, $z$ has a direct relationship with the number of samples, again for a given probability of conversion. The figure shows this graphically.

*We’re given a fixed value for the conversion rates of the A/B group, the relationship between the number of users in the A/B groups, and the z value. Assuming the conversion rate wouldn’t change as we collected more data, we’d need around 3,000 users in each group to hit a confidence interval of 70%. This rises to around 5,000 per group for 80%, 7,500 for 90%, and 12,000 for 95%.*

![image](images/07fig02_alt.jpg)


What you see is that for a given conversion rate of the two groups, the more users in the test groups, the more confident we become of the alternative hypothesis. Intuitively, this makes sense: we should expect to become more confident the more data we collect! Note that we can also draw a similar diagram holding the number of users constant and varying the difference between the groups. We must note, however, that for the type of applications we’re looking at, we shouldn’t expect large magnitude changes in performance. This may indicate that we must collect a lot of data to be sure a change does lead to a significant improvement in performance. The code to generate the graph in figure is in a separate file; try generating a similar one with the difference in conversion rates across the x-axis instead!

### Suitability of A/B

In this section, we’ve given a basic introduction to a statistical method for testing called the z-test. We discussed it in the context of an A/B test, where we want to test the impact of a change in operations so as to decide whether to adopt this change permanently. What we discovered is that for very small changes in performance, similar to those that may be typical of intelligent web applications, we need to create quite large control and test groups. In the case of our imaginary retailer, depending on the number of users seen daily, it may take considerable time to reach a significant conclusion. The problem with this in a business setting is that while you’re running the test, you’re potentially running suboptimally because either half of your users are in an under-performing test set or half of your users are in an under-performing control set, and you have to wait for the test to finish!

This is a classic problem known as the *explore-exploit conundrum*. We need to run suboptimally to explore the space and find solutions that perform better, but once we find these solutions, we also need to exploit them as soon as possible in order to gain an advantage. What if we could exploit new solutions in a quicker fashion, without waiting for the test to finish completely? Well, we can. Enter the multi-armed bandit (MAB).

## Multi-armed bandits

The name _multi-armed bandit_ (_MAB_) comes from the popular casino game, the one-armed bandit. For those of you who have never been to a casino, this is a machine whereby you pull on a lever (the arm); depending on the values displayed by the machine, you’re either paid out or (more likely) not. As you’d expect, the odds of these machines are in favor of the house, so the likelihood of being paid out is typically very small!

The (theoretical) MAB extends this formalism by imagining that you’re faced with a bank of one-armed bandits, each of which is assigned to pay out with an independent probability. As a player, you’re unaware of the probabilities of payout behind these machines, and the only way you can find out is by playing them. You’re tasked with playing these machines and maximizing your reward. So what strategy should you use? The figure illustrates the MAB formalism.

*The multi-armed bandit problem. A user is faced with a number of one-armed bandits, each of which has a different probability of paying out. The user doesn’t know what these probabilities are and may only uncover them through playing the machines. What strategy should the user employ in order to maximize their return? The user must explore the space in order to determine the probabilities of paying out, but they must also exploit machines with high probabilities of paying out in order to maximize their return.*

![image](images/07fig03.jpg)

### Multi-armed bandit strategies

Let’s define the problem more rigorously before we dive into some code! Formally, we have $k$ bandits, with the observable probabilities of paying out equal to $p_k$. We assume that only one arm can be pulled at a time and that the machine either pays out or doesn’t, with a value based on its associated probability. This is a finite game, with a set number of plays allowed. At any particular point in the game, the horizon $H$ is defined as the number of remaining plays that are permissible.

The user attempts to maximize the payout over all the machines. At any given point in the game, we can assess how well the user is doing through the use of a metric known as the _regret_. This is the difference between the payout that would be received if the user had an oracle and chose the optimal bandit at every step and the actual payout so far. Formally, the regret is defined as

![image](images/169equ01.jpg)

where $T$ is the number of steps we’ve carried out so far, $r_t$ is the reward that was received at step $t$, and $μ_{opt}$ is the mean payout returned per play from the optimal bandit. The lower the value of the regret, the more optimal the strategy is. But because this metric is susceptible to chance (payouts may be greater than the expected payout playing the best bandit alone), we may choose to use the expected regret instead. Formally, the expected regret is defined as

![image](images/170equ01.jpg)


where $μ_t$ is the (unobservable) mean payout from the arm chosen at time $t$. Because the second term is the expected payout from the selected strategy, it will be less than or equal to the expected payout from the optimal strategy, choosing the arm with $μ_{opt}$ all the time.

In the following sections, we’ll introduce a new variable to our formalism: epsilon. *Epsilon* controls the trade-off between exploring the space and exploiting the best-known solution, as you’ll see in the following strategies. It’s expressed as a probability.

#### Epsilon First

*Epsilon first* is the simplest of the MAB strategies that can be employed and can be considered equivalent to the A/B approach outlined earlier. Given $ϵ$, we perform exploration (1 – $ϵ$) × $N$ times, where $N$ is the total number of trials available in the game. The remaining trials are purely exploitative.

The `update_best_bandit` method keeps a running total of the payouts for each bandit and the number of times that bandit was played. The `best_bandit` variable is updated after every play to contain the index of the bandit that so far has the best payout rate. The following pseudo-code outlines the solution.

```python
    epsilon=0.1 
    best_bandit #index of the best bandit in the array
    bandit_array #array containing bandit objects
    total_reward=0
    number_trials
    current_trial=0
    number_explore_trials = (1-epsilon)*number_trials
    while((number_trials-current_trial)>0):
        if(current_trial<number_explore_trials):
            random_bandit = rand(0,len(bandit_array))
            total_reward += play(bandit_array[random_bandit])
            update_best_bandit()#update the best bandit
        else:
            total_reward +=play(bandit_array[best_bandit])
        current_trial+=1
```

#### Epsilon Greedy

With the _epsilon-greedy_ approach, $ϵ$ acts as a probability that we’ll explore the space, as opposed to exploiting the best-known arm. More formally, the following pseudo-code outlines the approach.

```python
epsilon=0.1 
best_bandit
bandit_array
total_reward=0
number_trials
current_trial=0
while((number_trials-current_trial)>0):
    random_float = rand(0,1)
    if(random_float<epsilon):
        random_bandit = rand(0,len(bandit_array))
        total_reward += play(bandit_array[random_bandit])
        update_best_bandit()
    else:
        total_reward +=play(bandit_array[best_bandit])
    current_trial+=1

```

The advantage of this approach is that we don’t need to wait for the explore phase to complete before we can start exploiting our knowledge of the bandit’s performance—but be careful! This algorithm doesn’t take into account the statistical significance of our performance data. It’s possible that a peak in positive payouts for a particular bandit will result in all plays being shifted to this bandit erroneously. More on this shortly.

You can see that $ϵ$ controls the probability that we explore rather than exploit. Low values of `array` make it less likely that we’ll explore the space, whereas higher values make it more likely. There’s a clear trade-off here, and the selection of our value of `ϵ` depends on many factors. Both the number of bandits and the probabilities of payouts will affect the regret, as introduced previously. One clear issue is that of bootstrapping. At the start of the experiment, we know nothing about the performance of any of the bandits (unlike in the epsilon-first strategy). Might there be a better way to do this, exploring the space more where the horizon is far away, decreasing as the horizon gets closer?

#### Epsilon Decreasing

The *epsilon-decreasing* strategy does just this. At the start of the experiment, we have a high value of $ϵ$, so the likelihood of exploration is high. This value is gradually decreased as the horizon goes on, making the likelihood of exploitation higher, as shown in the following listing.

```python
epsilon=1
best_bandit 
bandit_array 
total_reward=0
number_trials
current_trial=0
while((number_trials-current_trial)>0):
    random_float = rand(0,1)
    if(random_float<epsilon):
        random_bandit = rand(0,len(bandit_array))
        total_reward += play(bandit_array[random_bandit])
        update_best_bandit()
    else:
        total_reward +=play(bandit_array[best_bandit])
    current_trial+=1
    epsilon = update_epsilon(epsilon)

```

Note that there are several methods to choose an optimal rate to update $ϵ$. This depends on the number of bandits, $N$, and the respective weights at which they pay out.

#### Bayesian Bandits

As mentioned, one of the limitations of this algorithm is that we don’t take into account the significance of the performance data as we explore the space. Although you might be able to exploit your knowledge sooner, you may be making erroneous decisions. That’s not to say that these techniques aren’t useful! But they require a careful eye for the parameters, to ensure that they don’t exploit too early or explore for too long.

Enter *Bayesian bandits*. Similarly to A/B testing, we assume that the payout rate for each bandit is modeled as a distribution over payout rates. When we start out, each bandit has a very general prior (because any payout rate is equally likely for that bandit). The more we play a bandit, the more information we have about its performance, so we update its payout distribution over likely payout rates. When it comes to choosing which bandit to play, we sample from each distribution over the payout rates and choose the bandit that corresponds to the sample with the highest rate. The figure provides a graphical overview of the knowledge held about three illustrative bandits at a given time.

*Modeling the knowledge about the payout rate of three bandits using the Bayesian bandit method. The mean payout rates for arms 1, 2, and 3 are 0.1, 0.3, and 0.4, respectively. Bandit 1 has a lower mean but a much wider variance. Bandit 2 has a higher mean and smaller variance. Bandit 3 has an even higher mean and a smaller variance. In order to choose the bandit to play, each distribution is sampled, and the bandit corresponding to the distribution from which the highest sample was drawn will be picked. After selection, the bandit is played and the respective distribution is updated. This has the effect that even bandits with a low payout rate have a chance to redeem themselves if their mean rate is uncertain (high variance).*

![image](images/07fig04.jpg)

Here you can see that the knowledge about the distributions of our bandits is encoded as three distributions. Each distribution has an increasing mean, with decreasing variance. Thus, we’re less certain about the true rate of payout of the bandit with mean 0.1 and most certain about the bandit with an average payout of 0.4. Because the bandit to play is achieved by sampling each distribution, it’s possible that the arm corresponding to the distribution around 0.1 is pulled. This will occur when the samples from bandit 2 and bandit 3 are unusually low and the sample from bandit 1 is unusually large. The following listing provides the pseudo-code for this algorithm.

```python
bandit_distribution_array
total_reward=0
number_trials
current_trial=0
while((number_trials-current_trial)>0):
    sample_array = sample(bandit_distribution_array)
    best_bandit = index_of(max(sample_array))
    reward =play(bandit_array[best_bandit])
    total_reward+=reward
    current_trial+=1
    update_distribution_array(best_bandit,reward)

```

You can see the elegance of this solution. Although it’s simple to implement, this approach both models the uncertainly of our estimates and provides an excellent regret when compared to our previous approaches, as you’ll see in the following sections.

## A/B vs. the Bayesian bandit

So far in this notebook we’ve discussed several approaches to making the right choice. Much has been said in the community about MABs, and there has been a lot of hype about them—with many seeing them as a no-brainer alternative to A/B testing. After all, if you can test your choices while performing optimization, doesn’t that make more sense than waiting for statistical significance with an A/B test?

Well, yes and no. As with all machine-learning techniques, there is a set of trade-offs to understand, and you should be careful to apply the right approach in the right situation. The table provides some of the considerations you should be aware of when making the choice between a Bayesian bandit and an A/B test.

*Considerations of a Bayesian bandit vs. an A/B test. Remember, you should consider your application area and choose the most appropriate solution.*

![image](images/ab_table.png)

Let’s take our initial motivational case of website optimization and discuss the appropriateness of each solution with respect to convergence speed. First, note that any change in behavior is likely to be small (<0.01), and thus we already know the Bayesian bandit will take longer to converge than with larger impact improvements. If we now add multiple choices, so that we’re testing many landing pages in the same experiment, we’ll further impact the convergence speed. Now, what if the underlying distribution of user conversion is changing more quickly than this model can converge? For example, seasonal trends, sales, or other external factors may affect the underlying distribution that we’re assuming to be static. In such cases, it may pay dividends to think carefully and devise a statistically significant A/B test. This is not to say that Bayesian bandits aren’t useful in such cases (and indeed we know otherwise), but it’s imperative that you understand the characteristics of their execution. They’re not a silver bullet, and you can’t simply throw in as many tests as you can think of and expect the technique to perform as well as an oracle. Be careful!

## Extensions to multi-armed bandits

As with our previous section on deep learning, what’s so exciting about this area is that it’s teeming with up-to-date research. At the time of writing, this is a very active area of research, and we recommend that you familiarize yourself with the literature. In this section, we’d like to cover a few interesting developments in the MAB space. Further information and references to research can be found online.

### Contextual bandits

Contextual bandits are an important extension to the typical approach, because they allow you to encode additional information as an input that influences the strategy of the player. This lends itself to approaches whereby the bandits don’t pay out based on a fixed probability but also depend on the context or situation. The figure provides a graphical illustration of this paradigm.

*A contextual bandit. The payout probability isn’t fixed and is dependent on the context or situation. One way to think about this is that the player has assigned to them a vector of attributes at a given time, and, depending on those attributes, the payout probabilities of the machine will change. The best strategy must minimize the cumulative regret with this in mind.*

![image](images/07fig13.jpg)

Solutions to such a problem have immediate application in the advertising world. Earlier in this notebook, we discussed the use of bandits to choose the most suitable ad property, such as color or shape. Solutions to this problem will allow for the optimization of global ad properties but can’t take into account specifics of the user to whom we’re showing the ad. Progress in the field of contextual bandits may help us change this.

### Adversarial bandits

If you recall from previous sections, up until now we’ve made the implicit assumption that the payout distribution is always static. That is, these distributions don’t change as we play the bandits. Solutions to the adversarial bandits problem don’t make this assumption. In this formalism, the play proceeds as follows:

1.  The adversary selects a vector: the size of the number of bandits. This contains the rewards for each bandit at that step.

2.  The player, without knowledge of the adversary’s selection, chooses a bandit to play based on their strategy.

3.  In the full-information game, the player then may see the entire reward vector. In the partial-information game, the player sees only the reward associated with their chosen bandit.

Play continues in this way over a fixed number of steps, and the player must maximize their reward and minimize their regret. The figure provides a graphical overview of the problem.

*The adversarial bandit problem. In this variant of the MAB problem, solutions make no assumptions about the underlying distributions of the rewards. Instead, the problem is modeled as a game between a player and an adversary. At each time step, the adversary selects a reward vector before the player chooses an arm. In one variant of the game (full information), the player gets to see the full reward vector, and in another (partial information) the player sees the reward of their chosen bandit.*

![image](images/07fig14.jpg)

## Summary

- We covered several methods for making the right choice. Put simply, we looked at strategies for making a choice when faced with a multitude of options.
- We also covered the use of the z-score in an A/B test and discussed the implications of statistical significance.
- You’ve seen that the more data you collect, the more certain you can be about a potential change in performance.
- We demonstrated the other factors that impact significance: the variance of the two groups and the absolute differences in their rates.
- You were introduced to the concept of the multi-armed bandit. MABs allow us to use early information (before statistical significance is reached) to probabilistically modify the distribution of our choices.